In [8]:
import cv2
image = cv2.imread('../frames/kang0.jpg')
print(image.shape)
image = cv2.rectangle(image, (500, 0), (100, 1080), (0, 0, 255), 100)
cv2.imshow('img', image)
cv2.waitKey(0)
cv2.destroyAllWindows()